# unzip file: only need to do it once

In [ ]:
# unzip train.zip
# import zipfile as zf

# files = zf.ZipFile("train.zip",'r')
# files.extractall()
# files.close()

# load data

In [1]:
import glob
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
def input_data(name, Id_range = None, add_Date = False):
    files = glob.glob(r"C:\Users\Lenovo\Python_workshop\project\train\\"+name+"\\*.csv")
    if add_Date:
        dfs = []
        for f in files:
            df = pd.read_csv(f, sep=",")
            df['Date'] = f[-12:-4]
            df = df[df.Id.isin(Id_range)]
            dfs.append(df)
    else:
        dfs = [pd.read_csv(f, sep=",") for f in files]    
    data = pd.concat(dfs,ignore_index=True)
    data.Date = data.Date.apply(str)
    return data

In [3]:
ret = input_data("return")
ret.head()

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140


In [4]:
Id_range = ret.Id.unique()
risk = input_data("risk", Id_range = Id_range, add_Date = True)
risk.head()

,Id,estVol,Date
0,ID000BC6J11,0.158814,20140102
1,ID000BL7S92,0.119455,20140102
2,ID000BFGNH7,0.144518,20140102
3,ID000BW6697,0.167206,20140102
4,ID000BD9L75,0.162744,20140102


In [5]:
ret = pd.merge(
    ret,
    risk,
    how="left",
    on=['Date', 'Id'])

In [6]:
mdv = input_data("mdv")
mdv.head()

,Date,Id,MDV_63
0,20140102,ID000B9WTK6,3739.80
1,20140102,ID000B9ZG12,811845.80
2,20140102,IDZQ0000RP5,2562546.50
3,20140102,IDZQ00028F6,103370.21
4,20140102,IDZQ0003GF7,4321539.00


In [7]:
volume = input_data("price_volume")
volume.head()

,Date,Time,Id,CleanMid,CumVolume,IsOpen
0,20140102,10:00:00.000,IDZQ0003GF7,32.009570,30851.0,1.0
1,20140102,10:00:00.000,IDZQ00047M8,5.929723,100377.0,1.0
2,20140102,10:00:00.000,IDZQ00048R1,46.881290,118814.0,1.0
3,20140102,10:00:00.000,IDZQ0005R34,6.155510,57972.0,1.0
4,20140102,10:00:00.000,IDZQ0005SC2,27.234093,399523.0,1.0


In [8]:
shout = input_data("shout")
shout.head()

,Date,Id,SharesOutstanding
0,20140102,ID000B9WTK6,1.848
1,20140102,ID000B9ZG12,309.872
2,20140102,IDZQ0000RP5,402.458
3,20140102,IDZQ00028F6,126.594
4,20140102,IDZQ0003GF7,110.580


In [9]:
ret = pd.merge(
    ret,
    shout,
    how="left",
    on=['Date', 'Id'])

In [10]:
ret = pd.merge(
    ret,
    mdv,
    how="left",
    on=['Date','Id'])

In [11]:
ret = pd.merge(
    ret,
    volume,
    how="left",
    on=['Date','Time', 'Id'])

In [12]:
ret.head()

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0


# missing value

In [13]:
# get the number of missing data points per column
missing_values_count = ret.isnull().sum()
missing_values_count, ret.shape

(Date                              0
 Time                              0
 Id                                0
 ResidualNoWinsorCumReturn         0
 RawNoWinsorCumReturn              0
 estVol                            0
 SharesOutstanding                84
 MDV_63                            6
 CleanMid                     254473
 CumVolume                    254473
 IsOpen                       254473
 dtype: int64, (2448284, 11))

## dropna

In [14]:
# how many total missing values do we have?
total_cells = np.product(ret.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

# remove all the rows that contain a missing value
ret = ret.dropna()

2.8350432013160685


# fillna: haven't been run yet

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_ret_train = pd.DataFrame(my_imputer.fit_transform(ret_train))
imputed_ret_valid = pd.DataFrame(my_imputer.transform(ret_valid))

# Imputation removed column names; put them back
imputed_ret_train.columns = ret_train.columns
imputed_ret_valid.columns = ret_valid.columns

# summary

In [15]:
ret.shape

(2193760, 11)

In [16]:
len(ret.Id.unique())

1251

In [17]:
date_range = ret.Date.unique()
len(date_range)

1026

# winsorization

In [18]:
import matplotlib.pyplot as plt

In [19]:
# extract numeric columns
numeric_columns = list(ret.dtypes[ret.dtypes != "object"].index)
numeric_columns.remove('IsOpen')
numeric_columns

['ResidualNoWinsorCumReturn',
 'RawNoWinsorCumReturn',
 'estVol',
 'SharesOutstanding',
 'MDV_63',
 'CleanMid',
 'CumVolume']

## cross-sectional winsorize

In [20]:
from statistics import median

In [33]:
for feature in numeric_columns:   
    ret[str(feature+' winsorized')] = np.zeros(ret.shape[0])
    #for date in ['20140102','20140103']:  
    for date in date_range:  
        # compute limits
        mad = ret[ret.Date==date][feature].mad()
        med = median(ret[ret.Date==date][feature])        
        llimit, ulimit = med - 5*mad, med + 5*mad
        
        # Create winsorized versions of the features with new column names        
        ret[str(feature+' winsorized')][ret.Date==date] = np.clip(ret[ret.Date==date][feature],llimit, ulimit)
        

In [34]:
ret.head()

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen,ResidualNoWinsorCumReturn winsorized,RawNoWinsorCumReturn winsorized,estVol winsorized,SharesOutstanding winsorized,MDV_63 winsorized,CleanMid winsorized,CumVolume winsorized
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0


In [31]:
ret.shape

(2193760, 18)

In [ ]:
ret_train = ret[~ret.Date.str.contains('2017')]
ret_test = ret[ret.Date.str.contains('2017')]

## all data winsorize: don't use

In [ ]:
for feature in numeric_columns:   
    plt.boxplot(ret_train[feature])
    plt.title(str(feature))
    plt.show()

In [ ]:
def winsorization(data1, data2, feature, percnt1, percnt2, plot = True):
    # Only use train data to choose winsorization limits for these features
    llimit = np.percentile(data1[feature],percnt1)
    ulimit = np.percentile(data1[feature],percnt2)
    
    # Perform winsorization on both training and test data
    # Create winsorized versions of the features with new column names
    data1[str(feature+' winsorized')] = np.clip(data1[feature], llimit, ulimit)
    data2[str(feature+' winsorized')] = np.clip(data2[feature], llimit, ulimit)
    
    # plot the histogram after winsorization
    if plot:
        plt.boxplot(data1[str(feature+' winsorized')])
        plt.title(str(feature+' winsorized'))
        plt.show()

In [ ]:
winsorization(ret_train, ret_test, 'ResidualNoWinsorCumReturn', 0.05, 99.5)

In [ ]:
winsorization(ret_train, ret_test, 'RawNoWinsorCumReturn', 0.05, 99.5)

In [ ]:
winsorization(ret_train, ret_test, 'SharesOutstanding', 0, 99.5)

In [ ]:
winsorization(ret_train, ret_test, 'CumVolume', 0, 99.5)